In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter("ignore")

df_a: df de amazon <br>
df_d: df de disney<br>
df_n: df de netflix<br>
df_h: df de hulu

In [3]:
ruta = '../Datos/archivos'
df_a= pd.read_csv(ruta+'/amazon_prime_titles.csv')
df_d= pd.read_csv(ruta + '/disney_plus_titles.csv')
df_n= pd.read_json(ruta + '/netflix_titles.json')
df_h= pd.read_csv(ruta + '/hulu_titles.csv')

In [4]:
#lista con los df para iterar en los distintos procesos
dfs = {
    "amazon": df_a,
    "disney": df_d,
    "netflix": df_n,
    "hulu": df_h
}

## Criterios

Los criterio de la limpieza y pre-porcesamiento de los datos lo hice en base a las consultas que quiero responder.

Las consultas a realizar son:

+ Máxima duración según tipo de film (película/serie), por plataforma y por año:
    El request debe ser: get_max_duration(año, plataforma, [min o season])

+ Cantidad de películas y series (separado) por plataforma
    El request debe ser: get_count_plataform(plataforma)  
  
+ Cantidad de veces que se repite un género y plataforma con mayor frecuencia del mismo.
    El request debe ser: get_listedin('genero')  
    Como ejemplo de género pueden usar 'comedy', el cuál deberia devolverles un cunt de 2099 para la plataforma de amazon.

+ Actor que más se repite según plataforma y año.
  El request debe ser: get_actor(plataforma, año)

### Analizo los datos que tengo:<br>
    - si el tipo de dato coincide con el Dtype
    - exitencia de conflicto entre los df
    - cantidad de valores faltantes

Los df comparten las mismas columnas.

In [5]:
dfs['amazon'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9668 entries, 0 to 9667
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       9668 non-null   object
 1   type          9668 non-null   object
 2   title         9668 non-null   object
 3   director      7585 non-null   object
 4   cast          8435 non-null   object
 5   country       672 non-null    object
 6   date_added    155 non-null    object
 7   release_year  9668 non-null   int64 
 8   rating        9331 non-null   object
 9   duration      9668 non-null   object
 10  listed_in     9668 non-null   object
 11  description   9668 non-null   object
dtypes: int64(1), object(11)
memory usage: 906.5+ KB


In [6]:
dfs['disney'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1450 entries, 0 to 1449
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       1450 non-null   object
 1   type          1450 non-null   object
 2   title         1450 non-null   object
 3   director      977 non-null    object
 4   cast          1260 non-null   object
 5   country       1231 non-null   object
 6   date_added    1447 non-null   object
 7   release_year  1450 non-null   int64 
 8   rating        1447 non-null   object
 9   duration      1450 non-null   object
 10  listed_in     1450 non-null   object
 11  description   1450 non-null   object
dtypes: int64(1), object(11)
memory usage: 136.1+ KB


In [7]:
dfs['netflix'].info()

<class 'pandas.core.frame.DataFrame'>
Index: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 894.5+ KB


In [8]:
dfs['hulu'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3073 entries, 0 to 3072
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   show_id       3073 non-null   object 
 1   type          3073 non-null   object 
 2   title         3073 non-null   object 
 3   director      3 non-null      object 
 4   cast          0 non-null      float64
 5   country       1620 non-null   object 
 6   date_added    3045 non-null   object 
 7   release_year  3073 non-null   int64  
 8   rating        2553 non-null   object 
 9   duration      2594 non-null   object 
 10  listed_in     3073 non-null   object 
 11  description   3069 non-null   object 
dtypes: float64(1), int64(1), object(10)
memory usage: 288.2+ KB


La columna 'cast' tiene Dtype float64, cuando debería ser object.
Además que no tiene datos por lo que cambiare los ceros a 'Sin datos'

### Conflicto en show_id

In [ ]:
for plataforma, df in dfs.items():
    print(f'Primero 10 elementos de la columna show_id de {plataforma}: {df["show_id"].values[:10]}')

Primero 10 ids de amazon: ['s1' 's2' 's3' 's4' 's5' 's6' 's7' 's8' 's9' 's10']
Primero 10 ids de disney: ['s1' 's2' 's3' 's4' 's5' 's6' 's7' 's8' 's9' 's10']
Primero 10 ids de netflix: ['s1' 's2' 's3' 's4' 's5' 's6' 's7' 's8' 's9' 's10']
Primero 10 ids de hulu: ['s1' 's2' 's3' 's4' 's5' 's6' 's7' 's8' 's9' 's10']


- Los show_id de las ditintas plataformas se repiten por los que para evitar conficiones le agrego la letres de la plataforma delante del número. Ademas le quito la 's' que tiene delante del id.

### Doy formato a los datos que estan mal

In [22]:
for plataforma, df in dfs.items():
    print(f'Primero 10 elementos de la columna duration de {plataforma}: {df["duration"].values[:10]}')

Primero 10 elementos de la columna duration de amazon: ['113 min' '110 min' '74 min' '69 min' '45 min' '52 min' '98 min'
 '131 min' '87 min' '92 min']
Primero 10 elementos de la columna duration de disney: ['23 min' '91 min' '23 min' '41 min' '1 Season' '94 min' '1 Season'
 '2 Seasons' '2 Seasons' '45 min']
Primero 10 elementos de la columna duration de netflix: ['90 min' '2 Seasons' '1 Season' '1 Season' '2 Seasons' '1 Season'
 '91 min' '125 min' '9 Seasons' '104 min']
Primero 10 elementos de la columna duration de hulu: [nan '94 min' '108 min' '97 min' '104 min' '1 Season' '88 min' '1 Season'
 '1 Season' '3 Seasons']


- La columna 'duration' expresa la duracion de las peliculas en min o la cantidad de de temporadas si es una serie. <br>
Como es una varialbe cuantitativa discreta le cambio el Dtye a int64, para ello tengo que eliminar el 'min' y 'Season' o 'Seasons'

In [23]:
for plataforma, df in dfs.items():
    print(f'Tipo de la columna duration de {plataforma}: {df["duration"].dtype}')
    print(f'Tipo de la columna release_year de {plataforma}: {df["release_year"].dtype}')

Tipo de la columna duration de amazon: object
Tipo de la columna release_year de amazon: int64
Tipo de la columna duration de disney: object
Tipo de la columna release_year de disney: int64
Tipo de la columna duration de netflix: object
Tipo de la columna release_year de netflix: int64
Tipo de la columna duration de hulu: object
Tipo de la columna release_year de hulu: int64


Cambio el Dtype de la columna 'duration' y 'release_year' a uint16 por ser valores positivos.

# Normalizacion

In [14]:
def conseguir_val_unicos(df,col):
    '''
    df: dataframe 
    col: str del nimbre de la columna del df 

    Devulve los valores unicos de la columna en formato Serie de pandas.
    '''

    valores=[]
    lista_valores = []
    valores_df = df[col].values
    
    #llena lista_valores con listas del reparto de cada pelicula
    for val in valores_df:
        if type(val) == str:
            
            lista_valores.append(str.split(val,','))

    #llena valores con los actores
    for lista in lista_valores:
        for val in lista:
            valores.append(val.lstrip())#utlizo ltrip porque hay valores con un espacio delante
    
    #convierte la lista en una serie de pandas y elimina los repetidos
    valores = pd.Series(valores)
    valores = valores.drop_duplicates()
    return valores


In [24]:
def valores_col(lista_df,col):
    '''
    lista_df: lista de dataframes
    col: str

    Devuelve los valores unicos de todos los dataframes en lista_df de la columna col
    '''
    #serie vacia
    valores_col=pd.Series()

    #llena valores_col 
    for df in lista_df:
        valores_col=pd.concat([valores_col, conseguir_val_unicos(df,col)])
    
    # elimino los valores repetidos
    valores_col.drop_duplicates(inplace=True)
    #ordeno la lista
    valores_col.sort_values(inplace=True)
    return valores_col

 ### Normalizando la columna 'listed_in'

Listo las distintas categorias de los dataframes

In [25]:
categoria =  valores_col([df_a,df_d,df_n,df_h],'listed_in')

In [26]:
categoria.values

array(['Action', 'Action & Adventure', 'Action-Adventure',
       'Adult Animation', 'Adventure', 'Animals & Nature', 'Animation',
       'Anime', 'Anime Features', 'Anime Series', 'Anthology', 'Arthouse',
       'Arts', 'Biographical', 'Black Stories', 'British TV Shows',
       'Buddy', 'Cartoons', 'Children & Family Movies',
       'Classic & Cult TV', 'Classic Movies', 'Classics', 'Comedies',
       'Comedy', 'Coming of Age', 'Concert Film', 'Cooking & Food',
       'Crime', 'Crime TV Shows', 'Cult Movies', 'Dance', 'Disaster',
       'Documentaries', 'Documentary', 'Docuseries', 'Drama', 'Dramas',
       'Entertainment', 'Faith & Spirituality', 'Faith and Spirituality',
       'Family', 'Fantasy', 'Fitness', 'Game Show / Competition',
       'Game Shows', 'Health & Wellness', 'Historical', 'History',
       'Horror', 'Horror Movies', 'Independent Movies', 'International',
       'International Movies', 'International TV Shows', 'Kids',
       "Kids' TV", 'Korean TV Shows', 'LGBTQ'

Esta seria la normalizacion para categorias

In [28]:
valores_normalizar ={'Action & Adventure':['Action-Adventure','TV Action & Adventure'],
                    'Anime':['Animation','Anime', 'Anime Features', 'Anime Series'],
                    'Classic':['Classic & Cult TV', 'Classic Movies', 'Classics'],
                    'Comedy':['TV Comedies','Comedies'], 'Crime':['Crime TV Shows'], 'Documentary':['Documentaries'],
                    'Drama':['TV Dramas', 'Dramas'],'Faith & Spirituality':['Faith and Spirituality'],'Game Shows':['Game Show / Competition'],
                    'History':['Historical'], 'Horror': ['TV Horror','Horror Movies'],'International':['International Movies', 'International TV Shows'],
                    'Kids':["Kids' TV"],'LGBTQ':['LGBTQ Movies', 'LGBTQ+'],'Lifestyle':['Lifestyle & Culture'],'Mystery':['TV Mysteries'],
                    'Music':['Music & Musicals', 'Music Videos and Concerts', 'Musical'],'Reality':[ 'Reality TV'],
                    'Romance':['Romantic Movies','Romantic TV Shows'], 'Sports':['Sports Movies'],'Sci-Fi & Fantasy':['TV Sci-Fi & Fantasy'],
                    'Stand Up':['Stand-Up Comedy','Stand-Up Comedy & Talk Shows'],'Thriller':['TV Thrillers','Thrillers']
                   }

# Lleno los datos faltantes con datos de las distintas plataformas

Los valores de inteser segun los criterio ya mencionados son:<br>
- duration(netflix:3,hulu:479)
- cast(amazon:1233,disney:190,netflix:825,hulu:3037)
- realise_year(completo)
- list_in(completo)

Netflix y Hulu tienen valores faltantes en 'duration' y 'cast' uso la funcion completar_df

A Dysney+ y Amazon solo falta completar la columna cast 

## Ultimas modificaciones

Elimino las columnas 'director','country','date_added','rating' y 'description' que no aporta a las query requeridas.

Lleno los valores faltantes con 'sin datos'

Pongo en minuscula los valores

Elimino los espacios al principio y final de los valores